# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from API_keys import Google_API_Key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ushuaia,75,AR,1603803259,77,-54.80,-68.30,5.00,10.30
1,1,mahebourg,40,MU,1603803272,61,-20.41,57.70,26.00,4.10
2,2,lima,75,PE,1603803493,87,-12.04,-77.03,17.22,3.10
3,3,yahotyn,0,UA,1603803544,62,50.28,31.76,15.00,5.00
4,4,kodiak,1,US,1603803213,56,57.79,-152.41,7.00,6.70
5,5,ahipara,21,NZ,1603803288,92,-35.17,173.17,14.67,1.04
6,6,constitucion,100,CL,1603803037,68,-35.33,-72.42,12.76,1.96
7,7,sitka,75,US,1603803293,81,57.05,-135.33,8.00,1.50
8,8,busselton,100,AU,1603803260,67,-33.65,115.33,18.33,4.28
9,9,cape town,0,ZA,1603803166,56,-33.93,18.42,21.11,8.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=Google_API_Key)
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]
fig = gmaps.figure (center=(50.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = 100, point_radius=3)
fig.add_layer(heat_layer)
fig
max_intensity

97

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
new_types_df = pd.DataFrame(weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
print(new_types_df)
max_temp = (new_types_df["Max Temp"] > 20)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] > 40
new_types_df[max_temp & wind_speed & cloudiness]


                     City  Max Temp  Wind Speed  Cloudiness
0                 ushuaia      5.00       10.30          75
1               mahebourg     26.00        4.10          40
2                    lima     17.22        3.10          75
3                 yahotyn     15.00        5.00           0
4                  kodiak      7.00        6.70           1
5                 ahipara     14.67        1.04          21
6            constitucion     12.76        1.96         100
7                   sitka      8.00        1.50          75
8               busselton     18.33        4.28         100
9               cape town     21.11        8.20           0
10               khatanga    -15.81        5.67         100
11                  kapaa     25.89        1.74         100
12                 castro     21.13        3.71          71
13             port keats     27.17        1.66          11
14            clyde river    -12.98        1.72          41
15                rikitea     23.90     

,City,Max Temp,Wind Speed,Cloudiness
11,kapaa,25.89,1.74,100
12,castro,21.13,3.71,71
15,rikitea,23.90,8.89,100
16,alofi,22.00,3.10,47
17,hilo,23.00,1.50,90
34,puerto escondido,25.79,1.64,100
35,emilio carranza,23.97,2.98,98
36,tual,28.36,4.37,59
37,butaritari,27.28,3.72,100
41,labuhan,27.85,1.58,100


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : Google_API_Key}

In [12]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index} : {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try: 
        print(f"The closest hotel in {city_name} is {results[0] ['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0] ['name']
    except (KeyError, IndexError):
        print("result missing...skipping.")
        print("-----------------------")
        time.sleep(1)
print("----------------End of Search------------------------")

----------------End of Search------------------------


In [13]:
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))